# Capstone Project: Singapore HDB Resale Price Prediction
___

<p align = 'center'>
  <img src = "https://github.com/ElangSetiawan/sg-hdb-resale/blob/main/images/hdb_shintaro_tay_st_photo.jpg?raw=true" width = 75%>
<p/>
Source : https://www.straitstimes.com/singapore/housing/households-that-received-help-with-mortgage-payments-nearly-triple-that-of-same


**Problem Statement**

Public housing in Singapore is subsidised housing built and managed by the government under the Housing and Development Board (HDB). Most public housing in Singapore is owner-occupied. Under Singapore’s housing ownership programme, housing units are sold to applicants who meet certain income, citizenship and property ownership requirements, on a 99-year leasehold. The estate’s land and common areas continue to be owned by the government. Owner-occupied public housing can be sold to others in a resale market, subject to certain restrictions. Prices within the resale market are not regulated by the government.

Demand for resale flats since the end of the Circuit Breaker has pushed prices and sales to new highs. According to the HDB Price Index in Q2 2021, resale flat prices climbed 3% from Q1 2021, growing for the fifth consecutive quarter since Q2 2020. Prices were also 11% higher compared to a year ago. As data scientists, we want to understand the factors driving the price of resale flats as and provide predicted sale price for property portals.

**Model Explored**

|Models|Description|
|---|---|
|LinearRegression|
|XGBRegressor|


**Evaluation Metrics**

The evaluation metrics will be overfitting/underfitting of less than 2% between train and test data.

**Workflow Process**  
1. Notebook 1 of 2 : General EDA
2. Notebook 1 of 2 : Geolocation preprocessing


**Data Sources**  
1. Singapore postal sector and districts: 
   https://lengandalbertproperty.com/singapore-district-code-and-district-map/#:~:text=How%20many%20Postal%20Districts%20are%20there%20in%20Singapore%3F,the%20first%20two%20digit%20of%20Singapore%20postal%20codes.
2. Singapore HDB information and resale prices
   https://data.gov.sg
3. Singapore primary schools
   https://en.wikipedia.org/wiki/List_of_schools_in_Singapore
4. Singapore MRT
   https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations
5. Singapore LRT
   https://en.wikipedia.org/wiki/List_of_Singapore_LRT_stations
6. Singapore Shopping Malls
   https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore

In [ ]:
# # installing less common packages (uncomment if you do not have these installed)
# !pip install geopy
# !pip install geopandas
# !pip install featuretools

In [14]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from mpl_toolkits import mplot3d
import seaborn as sns

import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
from geopy.distance import geodesic

import json 
import requests
import time

import datetime as dt

import shapely
from shapely import geometry
from shapely import ops
from shapely.geometry import Point, LineString, Polygon, MultiPoint
from shapely.ops import nearest_points

import warnings
warnings.filterwarnings('ignore')

sns.set_style('ticks')

pd.set_option('display.max_columns', None)

%matplotlib inline

# 1.0 Data Import
___

In [8]:
# HDB flat information - Location information
df_raw_property_info = pd.read_csv('../data/raw/hdb-property-information.csv')
print(df_raw_property_info.shape)
print(df_raw_property_info.columns)

(12442, 24)
Index(['blk_no', 'street', 'max_floor_lvl', 'year_completed', 'residential',
       'commercial', 'market_hawker', 'miscellaneous', 'multistorey_carpark',
       'precinct_pavilion', 'bldg_contract_town', 'total_dwelling_units',
       '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold',
       'exec_sold', 'multigen_sold', 'studio_apartment_sold', '1room_rental',
       '2room_rental', '3room_rental', 'other_room_rental'],
      dtype='object')


In [9]:
# Points of Interest
# MRT and LRT locations
df_raw_mrt_lrt = pd.read_csv('../data/raw/Singapore_MRT_LRT_stations.csv')
print(df_raw_mrt_lrt.shape)
print(df_raw_mrt_lrt.columns)

(167, 2)
Index(['station_id', 'station_name'], dtype='object')


In [11]:
# Primary School locations
df_raw_schools = pd.read_csv('../data/raw/Singapore_Primary_schools.csv')
print(df_raw_schools.shape)
print(df_raw_schools.columns)

(191, 1)
Index(['name'], dtype='object')


In [12]:
# Shopping mall locations
df_raw_malls = pd.read_csv('../data/raw/Singapore_Shopping.csv')
print(df_raw_malls.shape)
print(df_raw_malls.columns)

(155, 1)
Index(['name'], dtype='object')


In [19]:
# Mapping of postcode to districts
def keystoint(x):
    return {int(k): int(v) for k, v in x}
with open('../data/raw/Singapore_districts.json') as d:
    sg_districts = json.load(d, object_pairs_hook=keystoint)
    print(sg_districts)
    print(type(sg_districts))


{1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 2, 8: 2, 14: 3, 15: 3, 16: 3, 9: 4, 10: 4, 11: 5, 12: 5, 13: 5, 17: 6, 18: 7, 19: 7, 20: 8, 21: 8, 22: 9, 23: 9, 24: 10, 25: 10, 26: 10, 27: 10, 28: 11, 29: 11, 30: 11, 31: 12, 32: 12, 33: 12, 34: 13, 35: 13, 36: 13, 37: 13, 38: 14, 39: 14, 40: 14, 41: 14, 42: 15, 43: 15, 44: 15, 45: 15, 46: 16, 47: 16, 48: 16, 49: 17, 50: 17, 81: 17, 51: 18, 52: 18, 53: 19, 54: 19, 55: 19, 82: 19, 56: 20, 57: 20, 58: 21, 59: 21, 60: 22, 61: 22, 62: 22, 63: 22, 64: 22, 65: 23, 66: 23, 67: 23, 68: 23, 69: 24, 70: 24, 71: 24, 72: 25, 73: 25, 77: 26, 78: 26, 75: 27, 76: 27, 79: 28, 80: 28}
<class 'dict'>


In [25]:
print(int("554771")//10000, sg_districts[int("554771")//10000])


55 19


In [2]:
# HDB flat information - Resale Prices
df_raw_1990 = pd.read_csv('../data/raw/resale-flat-prices-based-on-approval-date-1990-1999.csv')
df_raw_2000 = pd.read_csv('../data/raw/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
df_raw_2012 = pd.read_csv('../data/raw/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
df_raw_2015 = pd.read_csv('../data/raw/resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv')
# Coverage up to 2021-11-23 - data.gov.sg
df_raw_2017 = pd.read_csv('../data/raw/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')

df_raw_1990.shape,df_raw_2000.shape,df_raw_2012.shape,df_raw_2015.shape,df_raw_2017.shape

((287196, 10), (369651, 10), (52203, 10), (37153, 11), (113753, 11))

# 2.0 General EDA
___

In [3]:
# getting some basic information about each dataframe
# shape of dataframe i.e. number of rows and columns
# total number of rows with null values
# total number of duplicates
# data types of columns

def basic_eda(df, df_name):
    print(df_name.upper())
    print()
    print(f"Rows: {df.shape[0]} \t Columns: {df.shape[1]}")
    print()
    
    print(f"Total null rows: {df.isnull().sum().sum()}")
    print(f"Percentage null rows: {round(df.isnull().sum().sum() / df.shape[0] * 100, 2)}%")
    print()
    
    print(f"Total duplicate rows: {df[df.duplicated(keep=False)].shape[0]}")
    print(f"Percentage dupe rows: {round(df[df.duplicated(keep=False)].shape[0] / df.shape[0] * 100, 2)}%")
    print()
    
    print(df.dtypes)
    print("-----\n")

In [4]:
dfs = [
    (df_raw_1990, 'from 1990 to 1999'),
    (df_raw_2000, 'from 2000 to 2012'),
    (df_raw_2012, 'from 2012 to 2014'),
    (df_raw_2015, 'from 2015 to 2017'),
    (df_raw_2017, 'from 2017 to 2021-11-23')
    ]

In [5]:
[basic_eda(df, name) for df, name in dfs]

FROM 1990 TO 1999

Rows: 287196 	 Columns: 10

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 1638
Percentage dupe rows: 0.57%

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price             int64
dtype: object
-----

FROM 2000 TO 2012

Rows: 369651 	 Columns: 10

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 1014
Percentage dupe rows: 0.27%

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
dtype: object
-----

FROM 2012 TO 2014

Rows: 52203 	 Colum

[None, None, None, None, None]

| Observations | Action |
|---|---|
|Duplicate rows detected | Investigate or remove duplicate rows |
|New data 'remaining_lease" is introduced from 2015 onwards | Create new column called remaining_lease for data prior to 2015|

In [66]:
# New column 'remaining_lease' was introduced from dataset of 2015 onwards.
# add column remaining_lease to dataframes of data prior to 2015, before concatenating all the datasets

df_raw_1990['remaining_lease'] = 'dummy'
df_raw_2000['remaining_lease'] = 'dummy'
df_raw_2012['remaining_lease'] = 'dummy'
df_all = pd.concat([df_raw_1990, df_raw_2000, df_raw_2012, df_raw_2015,df_raw_2017] )
df_all.reset_index(drop=True,inplace=True)
basic_eda(df_all, 'all_data')

ALL_DATA

Rows: 859956 	 Columns: 11

Total null rows: 0
Percentage null rows: 0.0%

Total duplicate rows: 3690
Percentage dupe rows: 0.43%

month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
dtype: object
-----



In [67]:
df_all.head(-1)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,dummy
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,dummy
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,dummy
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,dummy
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,dummy
...,...,...,...,...,...,...,...,...,...,...,...
859950,2021-11,YISHUN,EXECUTIVE,361,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,668000.0,65 years 08 months
859951,2021-11,YISHUN,EXECUTIVE,792,YISHUN RING RD,10 TO 12,144.0,Apartment,1987,690000.0,64 years 10 months
859952,2021-11,YISHUN,EXECUTIVE,611,YISHUN ST 61,10 TO 12,142.0,Apartment,1987,680000.0,65 years 01 month
859953,2021-11,YISHUN,EXECUTIVE,614,YISHUN ST 61,01 TO 03,142.0,Apartment,1987,632000.0,64 years 06 months


### 2.1 Basic feature engineering
___

| Observations | Action |
|---|---|
|Column 'month' is text yyyy-mm | Create new 'sale_date' column of datetime yyyy-mm-01 |
|Column 'remaining_lease" is text | Create new 'remaining_year' column of 99 - (sale_date.year - lease_commence_date)|

In [68]:
# New column 'sale_date' as datetime.

df_all['sale_date'] = pd.to_datetime(df_all['month']+'-01')
df_all['lease_date'] = pd.to_datetime(df_all['lease_commence_date'].astype(str), format='%Y')
df_all['flat_address'] = df_all['block']+' '+df_all['street_name']
df_all['remaining_year'] = 99 - (df_all.sale_date.dt.year - df_all.lease_date.dt.year)
df_all.head(-3)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,sale_date,lease_date,flat_address,remaining_year
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,dummy,1990-01-01,1977-01-01,309 ANG MO KIO AVE 1,86
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,dummy,1990-01-01,1977-01-01,309 ANG MO KIO AVE 1,86
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,dummy,1990-01-01,1977-01-01,309 ANG MO KIO AVE 1,86
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,dummy,1990-01-01,1977-01-01,309 ANG MO KIO AVE 1,86
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,dummy,1990-01-01,1976-01-01,216 ANG MO KIO AVE 1,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859948,2021-11,YISHUN,EXECUTIVE,387,YISHUN RING RD,04 TO 06,146.0,Maisonette,1988,688000.0,65 years 08 months,2021-11-01,1988-01-01,387 YISHUN RING RD,66
859949,2021-11,YISHUN,EXECUTIVE,328,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,668000.0,65 years 08 months,2021-11-01,1988-01-01,328 YISHUN RING RD,66
859950,2021-11,YISHUN,EXECUTIVE,361,YISHUN RING RD,01 TO 03,146.0,Maisonette,1988,668000.0,65 years 08 months,2021-11-01,1988-01-01,361 YISHUN RING RD,66
859951,2021-11,YISHUN,EXECUTIVE,792,YISHUN RING RD,10 TO 12,144.0,Apartment,1987,690000.0,64 years 10 months,2021-11-01,1987-01-01,792 YISHUN RING RD,65


### 2.2 Geocoding the flat addresses
___

In [104]:
def getcoordinates(address):
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    resultsdict = eval(req.text)
    if resultsdict['found']>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE'], resultsdict['results'][0]['POSTAL'], resultsdict['results'][0]['ADDRESS']
    else:
        return 0,0,0, address

In [106]:
lat, long, postcode, address = getcoordinates('389 YISHUN RING ROAD SINGAPORE')
print(lat,long,postcode,address)

0 0 0 389 YISHUN RING ROAD SINGAPORE


In [101]:
# Set up
df_address = df_all.copy()
df_address_location = pd.DataFrame(columns=['full_address','latitude','longitude','postcode'])

# Remove duplicate addresses from query to OneMap API
df_address.drop_duplicates(subset='flat_address', keep='first', inplace= True)

#Try small sample
address_list = list(df_address['flat_address'])[0:20]
result = {}
coordinates_list= []
count = 0
failed_count = 0
for address in address_list:
    result.latitude, result.longitude, result.postcode, result.full_address = getcoordinates(address)
    count = count + 1
    if (result.latitude == 0 && )
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(addresslist),'addresses')
        print(address)
        coordinates_list.append(None)
    if count%5 == 0:
        print('Processed ', count, ' addresses, ', failed_count, ' failed. Sleeping 1 second')
        time.sleep(1)

print('Total Number of Addresses processed:' , count)
print('Total Number of Addresses With No Coordinates',failed_count)

Processed  5  addresses,  0  failed. Sleeping 1 second
Processed  10  addresses,  0  failed. Sleeping 1 second
Processed  15  addresses,  0  failed. Sleeping 1 second
Processed  20  addresses,  0  failed. Sleeping 1 second
Total Number of Addresses processed: 20
Total Number of Addresses With No Coordinates 0


In [103]:
print(coordinates_list)

[(0, 0, 0, 'NOT FOUND'), ('1.36619678831054', '103.841505011903', '560216', '216 ANG MO KIO AVENUE 1 ANG MO KIO 22 SINGAPORE 560216'), ('1.369196965617', '103.841666636086', '560211', '211 ANG MO KIO AVENUE 3 SINGAPORE 560211'), ('1.36844644010937', '103.844516260527', '560202', '202 ANG MO KIO AVENUE 3 ANG MO KIO VIEW SINGAPORE 560202'), ('1.36682360872342', '103.83649123351', '560235', '235 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560235'), ('1.36834606813391', '103.837196046409', '560232', '232 ANG MO KIO AVENUE 3 KEBUN BARU PALM VIEW SINGAPORE 560232'), ('1.36569665029696', '103.844347269095', '563308', '308C ANG MO KIO AVENUE 1 TECK GHEE VISTA SINGAPORE 563308'), ('1.36558833593063', '103.840518883254', '560220', '220 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560220'), ('1.36598198004505', '103.840654039612', '560219', '219 ANG MO KIO AVENUE 1 ANG MO KIO GROVE SINGAPORE 560219'), (0, 0, 0, 'NOT FOUND'), ('1.36559244608528', '103.848386744168', '560320', '320 ANG MO 

In [88]:
df_address = df_all.copy()
df_address.drop_duplicates(subset='flat_address', keep='first', inplace= True)
list(df_address['flat_address'])[0:5]

['309 ANG MO KIO AVE 1',
 '216 ANG MO KIO AVE 1',
 '211 ANG MO KIO AVE 3',
 '202 ANG MO KIO AVE 3',
 '235 ANG MO KIO AVE 3']

In [71]:
# Save the dataframe as pickle. 
print("pickling df_all:", df_all.shape)
import pickle
picklefile = open('../data/interim/df_all.pickle', 'wb') #create a file
pickle.dump(df_all, picklefile, pickle.HIGHEST_PROTOCOL) #pickle the dataframe
picklefile.close() #close file


pickling df_all: (859956, 15)
